In [5]:
import os
import cv2
from tqdm import tqdm
import json
from xml.etree import ElementTree as ET

icdar_path = "E:/Datasets/Extracted/icdar17_POD/dataset"

anno_path = os.path.join(icdar_path, "Annotations")
img_path = os.path.join(icdar_path, "Images")

output_annotation_file = icdar_path + "/POD" + ".json"


#as per the dataset, train:0-960 and test:960-1600
train = range(960)
test = range(960,1600)

range(0, 960) range(960, 1600)


In [23]:
coco_data = {}
files = os.listdir(anno_path)    #get all files in the folder
cats = []

for i in tqdm(range(len(files))):
    xml_path = anno_path + '/' + files[i]
    xml = ET.parse(xml_path)
    
    root = xml.getroot()
    children = root.getchildren()
    
    for child in children:
        name = child.tag[:-6]
        if name not in cats:
            cats.append(name)
            
cats = sorted(cats)
         
    
list_dict = []
for i in range(len(cats)):
    tmp = {}
    tmp["supercategory"] = "none"
    tmp["id"] = i
    tmp["name"] = cats[i]
    list_dict.append(tmp)

coco_data["categories"] = list_dict
print(coco_data["categories"])

[{'supercategory': 'none', 'id': 0, 'name': 'figure'}, {'supercategory': 'none', 'id': 1, 'name': 'formula'}, {'supercategory': 'none', 'id': 2, 'name': 'table'}]


In [104]:
coco_data["images"] = []
coco_data["annotations"] = []

img_id = 0
ann_id = 0

for img_id in tqdm(train):
    
    xml_path = anno_path + '/' + files[img_id]
    xml = ET.parse(xml_path)
    
    root = xml.getroot()
    #get the name of file without extension
    fname = root.attrib['filename'][:-4]
    
    img_name = fname + ".bmp"
    img = img_path + '/' + img_name
    if(not os.path.isfile(img)):
        print('error reading file\n',img)
        continue
    img = cv2.imread(img, 1)
    h, w, c = img.shape

    images_tmp = {}
    images_tmp["file_name"] = img_name
    images_tmp["height"] = h
    images_tmp["width"] = w
    images_tmp["id"] = img_id
    coco_data["images"].append(images_tmp)
    

    children = root.getchildren()    
    for child in children:
        
        label = child.tag[:-6]
        coords = child.find('Coords')
        points = coords.attrib['points'].split(' ')
        x1,y1 = [int(p) for p in points[0].split(',')]
        x2,y2 = [int(p) for p in points[3].split(',')]

        annotations_tmp = {}
        annotations_tmp["id"] = ann_id
        ann_id += 1
        annotations_tmp["image_id"] = img_id
        annotations_tmp["segmentation"] = []
        annotations_tmp["ignore"] = 0
        annotations_tmp["area"] = (x2-x1)*(y2-y1)
        annotations_tmp["iscrowd"] = 0
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1]
        annotations_tmp["category_id"] = cats.index(label)

        coco_data["annotations"].append(annotations_tmp)

100%|████████████████████████████████████████| 960/960 [00:25<00:00, 38.26it/s]


In [105]:
outfile =  open(output_annotation_file, 'w')
json_str = json.dumps(coco_data, indent=4)
outfile.write(json_str)
outfile.close()